# Prepare mitosis time series data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models
from cellpose.io import imread
import glob
from pathlib import Path
from PIL import Image, ImageSequence
from tqdm import tqdm
import os
import os.path
# from livecell_tracker import segment
from livecell_tracker import core
from livecell_tracker.core import datasets
from livecell_tracker.core.datasets import LiveCellImageDataset, SingleImageDataset
from skimage import measure
from livecell_tracker.core import SingleCellTrajectory, SingleCellStatic

In [ ]:
sample_json_dir = Path("./EBSS_starvation_24h_xy16_annotation")
sample_dataset_dir = sample_json_dir / "datasets"
class_subfolders = ["mitosis", "apoptosis", "normal"]
# sample_paths = glob.glob(str(sample_json_dir / "*.json"))

class_samples = {}
for subfolder in class_subfolders:
    class_samples[subfolder] = []
    sample_paths = glob.glob(str(sample_json_dir / subfolder / "*.json"))
    for sample_path in sample_paths:
        sample = SingleCellStatic.load_single_cells_json(sample_path)
        class_samples[subfolder].append(sample)

In [ ]:
class_samples

## Prepare videos and annotations for MMDetection

In [ ]:
classes = class_samples.keys()
classes

In [ ]:
from livecell_tracker.core.utils import gray_img_to_rgb
from livecell_tracker.preprocess.utils import normalize_img_to_uint8

In [ ]:
def video_frames_masks_from_sample(sample):
    scs_by_time = {}
    for sc in sample:
        if sc.timeframe not in scs_by_time:
            scs_by_time[sc.timeframe] = []
        scs_by_time[sc.timeframe].append(sc)
    sc_times = sorted(scs_by_time.keys())
    window_size = 5
    sample_scs = []
    for i in range(window_size, len(sc_times)):
        time = sc_times[i]
        scs_at_time = scs_by_time[time]
        sample_scs.append(scs_at_time)

    # get the largest bounding box
    largest_bbox = [np.inf, np.inf, -np.inf, -np.inf]
    for scs in sample_scs:
        for sc in scs:
            bbox = sc.bbox
            if bbox[0] < largest_bbox[0]:
                largest_bbox[0] = bbox[0]
            if bbox[1] < largest_bbox[1]:
                largest_bbox[1] = bbox[1]
            if bbox[2] > largest_bbox[2]:
                largest_bbox[2] = bbox[2]
            if bbox[3] > largest_bbox[3]:
                largest_bbox[3] = bbox[3]

    # make largest_bbox coords integer
    largest_bbox = [int(x) for x in largest_bbox]

    video_frames = []
    video_frame_masks = []
    for scs in sample_scs:
        merged_label_mask = np.zeros((largest_bbox[2] - largest_bbox[0], largest_bbox[3] - largest_bbox[1]), dtype=np.uint8)
        tmp_img = scs[0].get_img_crop(bbox=largest_bbox)
        tmp_img = normalize_img_to_uint8(tmp_img)
        tmp_img = gray_img_to_rgb(tmp_img)
        for idx, sc in enumerate(scs):
            sc_label = idx + 1
            sc.bbox = largest_bbox
            sc_mask = sc.get_sc_mask(bbox=largest_bbox, dtype=int)

            # Warning: simply add the label masks will cause overlapping cells to generate unexpected labels
            _nonzero = sc_mask > 0
            merged_label_mask[_nonzero] = sc_label

            # # for debugging
            # fig, axes = plt.subplots(1, 2, figsize=(10, 5))
            # axes[0].imshow(tmp_img)
            # axes[1].imshow(merged_label_mask)
            # plt.show()
        video_frames.append(tmp_img)
        video_frame_masks.append(gray_img_to_rgb(normalize_img_to_uint8(merged_label_mask)))
    return video_frames, video_frame_masks

In [ ]:
import cv2
import numpy as np
import pandas as pd

def gen_mp4_from_frames(video_frames, output_file, fps=5):
    # Define the output video file name and properties
    frame_size = video_frames[0].shape[:2][::-1]  # reverse the order of width and height

    # Create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(output_file), fourcc, fps, frame_size)
    # Write each frame to the output video
    for frame in video_frames:
        out.write(frame)
    out.release()


sample = class_samples["mitosis"][3]
fps = 5
print("len sample: ", len(sample))
# record video file path and class label
annotation_df = pd
video_frames, video_frame_masks = video_frames_masks_from_sample(sample)
print("len video_frames: ", len(video_frames))
print("len masks video: ", len(video_frame_masks))

output_dir = Path('notebook_results/mmaction_train_data')
output_dir.mkdir(exist_ok=True)

output_file = output_dir / 'mitosis.mp4'
mask_output_file = output_dir / 'mitosis_masks.mp4'

gen_mp4_from_frames(video_frames, output_file, fps=fps)
gen_mp4_from_frames(video_frame_masks, mask_output_file, fps=fps)

